## 1. Código de scraping para o estatísticas dos jogadores no FBREF

In [ ]:
import pandas as pd
import time
import sqlite3
from datetime import datetime
import cloudscraper
from bs4 import BeautifulSoup, Comment
from io import StringIO
import re

def extrair_tabela_real_fbref(url):
    scraper = cloudscraper.create_scraper()
    try:
        response = scraper.get(url)
        if response.status_code != 200: return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        comentarios = soup.find_all(string=lambda text: isinstance(text, Comment))
        html_completo = response.text + "".join([str(com) for com in comentarios])
        
        tabelas = pd.read_html(StringIO(html_completo))
        for df in tabelas:
            cols = df.columns.get_level_values(-1) if isinstance(df.columns, pd.MultiIndex) else df.columns
            if 'Player' in cols:
                if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(0)
                df = df[df['Player'] != 'Player'].copy()
                return df.loc[:, ~df.columns.str.contains('^Unnamed')]
    except:
        return None
    return None

def scraper_fbref_liga_unica(url_base):
    # 1. Identificar ID e Nome da liga pela URL informada
    # Exemplo: https://fbref.com/en/comps/13/stats/Serie-A-Stats -> ID: 13, Nome: Serie-A
    match = re.search(r'comps/(\d+)/stats/(.+)-Stats', url_base)
    
    if not match:
        print("❌ Erro: O link fornecido não parece ser o link de 'Stats' (Standard) do FBRef.")
        return

    id_liga = match.group(1)
    nome_liga = match.group(2)
    
    categorias_slugs = [
        "stats", "shooting", "passing", "passing_types", "gca", 
        "defense", "possession", "playingtime", "misc", "keepers", "keepersadv"
    ]
    
    df_liga_completa = None
    conn = sqlite3.connect('scouting_sulamerica.db')

    # 2. Loop pelas categorias
    for cat in categorias_slugs:
        url_cat = f"https://fbref.com/en/comps/{id_liga}/{cat}/{nome_liga}-Stats"
        print(f"  -> Coletando {cat}...")
        
        df_temp = extrair_tabela_real_fbref(url_cat)
        
        if df_temp is not None:
            if df_liga_completa is None:
                df_liga_completa = df_temp
            else:
                cols_to_use = df_temp.columns.difference(df_liga_completa.columns).tolist()
                cols_to_use.extend(['Player', 'Squad'])
                df_liga_completa = pd.merge(df_liga_completa, df_temp[cols_to_use], on=['Player', 'Squad'], how='left')
                # Remove duplicatas de colunas que podem surgir no merge
                df_liga_completa = df_liga_completa.loc[:, ~df_liga_completa.columns.duplicated()].copy()
        
        # Respeita o servidor para não ser banido
        time.sleep(12) 

    # 3. Finalização e Salvamento
    if df_liga_completa is not None:
        df_liga_completa['League'] = nome_liga
        df_liga_completa['Data_Analise'] = datetime.now().strftime("%d/%m/%Y")
        
        # Limpa nomes de colunas para o SQL
        df_liga_completa.columns = [str(c).replace('/', '_per_').replace('.', '_').replace(' ', '_') for c in df_liga_completa.columns]
        
        # Nome da tabela dinâmico (ex: serie_b_player_stats)
        tabela_nome = f"{nome_liga.lower().replace('-', '_')}_player_stats"
        
        df_liga_completa.to_sql(tabela_nome, conn, if_exists='replace', index=False)
        conn.close()
        
        print(f"\n SUCESSO! Dados de '{nome_liga}' salvos na tabela '{tabela_nome}'.")
        print(f"Total de jogadores: {len(df_liga_completa)}")
    else:
        print(" Falha crítica: Nenhum dado pôde ser coletado.")

# --- MODO DE USO ---
# Basta trocar este link pela liga que você quer:
link_da_vez = "https://fbref.com/en/comps/105/stats/Venezuelan-Primera-Division-Stats" 

scraper_fbref_liga_unica(link_da_vez)


🚀 Iniciando captura da liga: Venezuelan-Primera-Division (ID: 105)
  -> Coletando stats...
  -> Coletando shooting...
  -> Coletando passing...
  -> Coletando passing_types...
  -> Coletando gca...
  -> Coletando defense...
  -> Coletando possession...
  -> Coletando playingtime...
  -> Coletando misc...
  -> Coletando keepers...
  -> Coletando keepersadv...

✅ SUCESSO! Dados de 'Venezuelan-Primera-Division' salvos na tabela 'venezuelan_primera_division_consolidado'.
📊 Total de jogadores: 494


## 2. Código de scraping para perfil dos jogadores (altura e pé de preferência) no Transfermarkt

In [ ]:
import pandas as pd
import cloudscraper
from bs4 import BeautifulSoup
from io import StringIO
import time
import sqlite3
import re

def extrair_links_times_automatico(url_liga):
    """Descobre os links dos times de qualquer liga fornecida."""
    scraper = cloudscraper.create_scraper()
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
    
    print(f"Buscando clubes da liga em: {url_liga}")
    response = scraper.get(url_liga, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    links_detalhados = []
    tabela = soup.find("table", {"class": "items"})
    
    if not tabela:
        print("Erro: Tabela de times não encontrada. Verifique se o link está correto.")
        return []

    for link in tabela.find_all('a', href=True):
        href = link['href']
        # Captura o padrão de link de time
        if '/startseite/verein/' in href:
            match = re.search(r'/(.+?)/startseite/verein/(\.?\d+)', href)
            if match:
                nome_slug = match.group(1)
                id_time = match.group(2)
                # URL detalhada (Kader + plus/1) para pegar Altura e Pé
                url_formatada = f"https://www.transfermarkt.com/{nome_slug}/kader/verein/{id_time}/plus/1"
                
                if url_formatada not in links_detalhados:
                    links_detalhados.append(url_formatada)
            
    return links_detalhados

def limpar_transfermarkt_detalhado(df):
    mapping = {
        'Player': [col for col in df.columns if 'Player' in col or 'Name' in col],
        'Height': [col for col in df.columns if 'Height' in col],
        'Foot': [col for col in df.columns if 'Foot' in col],
        'Value': [col for col in df.columns if 'Value' in col or 'Market' in col]
    }

    colunas_encontradas = {chave: lista[0] for chave, lista in mapping.items() if lista}
    df_limpo = df[list(colunas_encontradas.values())].copy()
    df_limpo.columns = list(colunas_encontradas.keys())

    # Remove as linhas de rótulo (Goalkeeper, Left, etc) que não têm altura
    if 'Height' in df_limpo.columns:
        df_limpo = df_limpo.dropna(subset=['Height'])

    # Limpeza de nomes (remove a posição que o TM gruda no texto)
    cargos = ['Goalkeeper', 'Centre-Back', 'Left-Back', 'Right-Back', 'Defensive Midfield', 
              'Central Midfield', 'Attacking Midfield', 'Left Winger', 'Right Winger', 
              'Second Striker', 'Centre-Forward', 'Defender', 'Midfield', 'Forward']
    
    for cargo in cargos:
        df_limpo['Player'] = df_limpo['Player'].str.replace(cargo, '', case=False).str.strip()

    # Remove números que ficam grudados no nome
    df_limpo['Player'] = df_limpo['Player'].str.replace(r'\d+', '', regex=True).str.strip()

    return df_limpo

def scraper_transfermarkt_automatico(url_liga_alvo):
    
    # 1. Identifica o nome da liga para salvar a tabela com nome correto
    match_nome = re.search(r'com/(.+?)/startseite', url_liga_alvo)
    nome_liga_limpo = match_nome.group(1).replace('-', '_') if match_nome else "liga_generica"
    
    links_times = extrair_links_times_automatico(url_liga_alvo)
    print(f"Total de times encontrados: {len(links_times)}")
    
    scraper = cloudscraper.create_scraper()
    todos_perfis = []

    for i, url in enumerate(links_times):
        nome_time_log = url.split('/')[3]
        print(f"[{i+1}/{len(links_times)}] Acessando: {nome_time_log}")
        
        try:
            response = scraper.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                tabela_html = soup.find('table', {'class': 'items'})
                
                if tabela_html:
                    df_raw = pd.read_html(StringIO(str(tabela_html)))[0]
                    df_limpo = limpar_transfermarkt_detalhado(df_raw)
                    df_limpo['Squad_TM'] = nome_time_log
                    todos_perfis.append(df_limpo)
            
            # Delay de segurança obrigatório para o TM
            time.sleep(15)
        except Exception as e:
            print(f"Erro em {nome_time_log}: {e}")

    if todos_perfis:
        df_final = pd.concat(todos_perfis, ignore_index=True)
        # Remove duplicatas finais por segurança
        df_final = df_final.drop_duplicates(subset=['Player', 'Squad_TM'])
        
        conn = sqlite3.connect('scouting_sulamerica.db')
        tabela_nome = f"{nome_liga_limpo}_perfil"
        df_final.to_sql(tabela_nome, conn, if_exists='replace', index=False)
        conn.close()
        print(f"\n Sucesso! Tabela '{tabela_nome}' criada com {len(df_final)} jogadores.")
    else:
        print("Nenhum dado coletado.")

# --- COMO USAR ---
# Cole aqui o link 'Startseite' da liga que você quer
link_liga = "https://www.transfermarkt.com/liga-futve-clausura/startseite/wettbewerb/VZ1C"

scraper_transfermarkt_automatico(link_liga)

Buscando clubes da liga em: https://www.transfermarkt.com/liga-futve-clausura/startseite/wettbewerb/VZ1C
Total de times encontrados: 14
[1/14] Acessando: carabobo-fc
[2/14] Acessando: deportivo-la-guaira
[3/14] Acessando: monagas-sc
[4/14] Acessando: universidad-central-de-venezuela
[5/14] Acessando: academia-puerto-cabello
[6/14] Acessando: caracas-fc
[7/14] Acessando: deportivo-tachira
[8/14] Acessando: academia-anzoategui-fc
[9/14] Acessando: metropolitanos-fc
[10/14] Acessando: zamora-fc
[11/14] Acessando: portuguesa-fc
[12/14] Acessando: estudiantes-de-merida
[13/14] Acessando: yaracuyanos-futbol-club
[14/14] Acessando: dvo-rayo-zuliano

✨ Sucesso! Tabela 'liga_futve_clausura_perfil' criada com 407 jogadores.


## 3. Código para Normalização, Limpeza dos dados e Geração da base de dados final

In [ ]:
import sqlite3
import pandas as pd
import unicodedata

def normalizar_nome(nome):
    if not nome: return ""
    nome = ''.join(c for c in unicodedata.normalize('NFD', str(nome)) if unicodedata.category(c) != 'Mn')
    return nome.lower().strip()

def limpar_valor_monetario(valor):
    if pd.isna(valor) or valor == '-' or valor == '': return 0.0
    v = str(valor).replace('€', '').replace(' ', '').replace(',', '.')
    try:
        if 'm' in v: return float(v.replace('m', '')) * 1_000_000
        if 'k' in v: return float(v.replace('k', '')) * 1_000
        return float(v)
    except: return 0.0

def gerar_base_scouting_final_total():
    conn = sqlite3.connect('scouting_sulamerica.db')
    
    # 1. Carregar Dados Brutos
    df_stats = pd.read_sql("SELECT * FROM venezuelan_primera_division_consolidado", conn) # Substitui com o nome da tabela de stats criado para determinada liga
    df_perfil = pd.read_sql("SELECT * FROM liga_futve_clausura_perfil", conn) # Substitui com o nome da tabela de perfil criado para determinada liga
    
    # 2. Preparar Chaves
    df_stats['nome_busca'] = df_stats['Player'].apply(normalizar_nome)
    df_stats['ID_FUSAO'] = df_stats['nome_busca'] + "_" + df_stats['Born'].astype(str) # Fusão dos jogadores por Nome + Ano de nascimento (foi utilizada como teste)

    # 3. DICIONÁRIO DE REGRAS
    regras_agrupamento = {}
    
    # Lista do que NUNCA deve ser somado (Identificação)
    cols_identificacao = ['Player', 'Nation', 'Pos', 'Squad', 'Born', 'Age', 'Data_Analise', 'nome_busca', 'ID_FUSAO']

    for col in df_stats.columns:
        # Se for coluna de texto ou idade/ano, mantemos o valor (last)
        if col in cols_identificacao:
            regras_agrupamento[col] = 'last'
        else:
            # Converte para número para garantir o cálculo
            df_stats[col] = pd.to_numeric(df_stats[col], errors='coerce')
            
            # Regra de Média: tudo que tem '90', '%' ou 'per'
            if any(x in col.lower() for x in ['90', '%', 'per']):
                regras_agrupamento[col] = 'mean'
            # Regra de Soma: Gols, Assistências, Minutos, Jogos
            else:
                regras_agrupamento[col] = 'sum'

    # 4. Agrupar e Fundir 
    df_stats_agrupado = df_stats.groupby('ID_FUSAO').agg(regras_agrupamento).reset_index(drop=True)

    # 5. Merge com Transfermarkt
    df_perfil['nome_busca'] = df_perfil['Player'].apply(normalizar_nome)
    df_perfil['Market_Value_Real'] = df_perfil['Value'].apply(limpar_valor_monetario)
    
    # Pega o melhor perfil do TM (com maior valor)
    df_perfil_unique = df_perfil.sort_values(by=['nome_busca', 'Market_Value_Real'], ascending=[True, False]).drop_duplicates(subset=['nome_busca'])

    df_final = pd.merge(
        df_stats_agrupado, 
        df_perfil_unique[['nome_busca', 'Height', 'Foot', 'Value', 'Market_Value_Real']], 
        on='nome_busca', 
        how='left'
    )

    # 6. Salvar e Printar
    df_final = df_final.drop(columns=['nome_busca', 'ID_FUSAO'])
    df_final.to_sql('base_scouting_venezuela', conn, if_exists='replace', index=False)
    conn.close()
    
    print(f"✅ Fusão Concluída: {len(df_final)} jogadores únicos.")
    return df_final

# Chamada do script
df_master = gerar_base_scouting_final_total()

✅ Fusão Concluída: 472 jogadores únicos.


## 4. Limpeza Final da base de dados

In [ ]:
def limpar_base_master_definitiva(df):
    df_clean = df.copy()

    # 1. Limpeza da Altura (Ex: "1,85m" -> 1.85)
    if 'Height' in df_clean.columns:
        df_clean['Height'] = (
            df_clean['Height']
            .str.replace('m', '', regex=False)
            .str.replace(',', '.', regex=False)
            .replace('-', None) # Transforma hífen em nulo real (NaN)
        )
        df_clean['Height'] = pd.to_numeric(df_clean['Height'], errors='coerce')

    # 2. Limpeza do Pé Preferido
    if 'Foot' in df_clean.columns:
        # Padroniza: 'left', 'right', 'both' ou None
        df_clean['Foot'] = df_clean['Foot'].replace('-', None).str.lower().str.strip()

    # 3. Limpeza do Valor de Mercado (Garante que a coluna Real seja numérica)
    if 'Market_Value_Real' in df_clean.columns:
        df_clean['Market_Value_Real'] = pd.to_numeric(df_clean['Market_Value_Real'], errors='coerce').fillna(0)

    # 4. Limpeza Geral de Estatísticas de Performance
    # Substitui NaNs por 0 em colunas que deveriam ser numéricas (Gols, Assistências, etc)
    cols_stats = df_clean.select_dtypes(include=['number']).columns
    df_clean[cols_stats] = df_clean[cols_stats].fillna(0)

    # 5. Organização de Colunas de Texto
    # Garante que não existam espaços sobrando nos nomes e posições
    for col in ['Player', 'Squad', 'Pos']:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str).str.strip()

    return df_clean

df_master = limpar_base_master_definitiva(df_master)

conn = sqlite3.connect('scouting_sulamerica.db')
df_master.to_sql('base_scouting_venezuela', conn, if_exists='replace', index=False) # Substituir o nome para a liga determinada
conn.close()

print("Base Master limpa e padronizada com sucesso!")

Base Master limpa e padronizada com sucesso!
